# 文件理解功能测试 - GLM-4.6V-Flash

本 Notebook 用于测试智谱 AI 的文件理解（File Understanding）功能。

> ⚠️ **重要提示**：目前仅 GLM-4.6V-Flash 免费模型支持文件理解功能，且不能与图片/视频同时传入。
>
> GLM-4.6V-Flash 支持思考模式（thinking 参数），启用后可获得思维链（reasoning_content）。

## 1. 环境配置

In [ ]:
import requests
import json
import os
from datetime import datetime

# API 配置
API_KEY = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"
API_URL = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
MODEL = "glm-4.6v-flash"

# 测试文件 URL
TEST_PDF_URL = "https://cdn.bigmodel.cn/static/demo/demo1.pdf"

print(f"模型: {MODEL}")
print(f"API 端点: {API_URL}")
print(f"测试时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# 通用调用函数
def chat_with_files(file_urls, question, model=MODEL):
    """
    调用文件理解 API
    
    Args:
        file_urls: 单个 URL 字符串或 URL 列表
        question: 问题文本
        model: 模型名称
    """
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    # 构建 content 列表
    content = []
    if isinstance(file_urls, str):
        file_urls = [file_urls]
    
    for url in file_urls:
        content.append({
            "type": "file_url",
            "file_url": {"url": url}
        })
    
    content.append({
        "type": "text",
        "text": question
    })
    
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": content
            }
        ]
    }
    
    try:
        response = requests.post(API_URL, json=payload, headers=headers, timeout=60)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

def print_response(response, title="响应"):
    """格式化打印响应结果"""
    print(f"\n{'='*60}")
    print(f"📋 {title}")
    print(f"{'='*60}")
    
    if "error" in response:
        print(f"❌ 错误: {response['error']}")
    elif "choices" in response:
        content = response["choices"][0]["message"]["content"]
        print(content)
        print(f"\n📊 Token 使用: {response.get('usage', {})}")
    else:
        print(json.dumps(response, indent=2, ensure_ascii=False))
    
    print(f"{'='*60}")

print("✅ 函数定义完成")

## 2. GLM-4.6V-Flash 单文件理解测试（PDF）

GLM-4.6V-Flash 支持思考模式，可通过 `thinking: {"type": "enabled"}` 启用思维链功能。

In [ ]:
# 测试 2.1: 基础文档分析
print("📝 测试 2.1: 基础文档分析")
response = chat_with_files(
    TEST_PDF_URL,
    "请分析这份文档的内容，简要说明这是什么类型的文档"
)
print_response(response, "基础文档分析")

In [ ]:
# 测试 2.2: 文档内容提取
print("📝 测试 2.2: 文档内容提取")
response = chat_with_files(
    TEST_PDF_URL,
    "请提取这份文档中的所有关键信息点，按条目列出"
)
print_response(response, "文档内容提取")

In [ ]:
# 测试 2.3: 文档结构分析
print("📝 测试 2.3: 文档结构分析")
response = chat_with_files(
    TEST_PDF_URL,
    "请分析这份文档的结构，说明包含哪些章节或部分"
)
print_response(response, "文档结构分析")

## 3. 多文件对比分析测试

In [ ]:
# 测试 3.1: 双文件对比（使用相同 PDF 两次模拟对比场景）
print("📝 测试 3.1: 多文件对比分析")
print("注意：由于只有一个测试文件，我们使用同一个文件两次来测试多文件功能")

file_urls = [TEST_PDF_URL, TEST_PDF_URL]
response = chat_with_files(
    file_urls,
    "这两份文档的内容是否一致？请详细对比说明"
)
print_response(response, "多文件对比分析")

In [ ]:
# 测试 3.2: 多文件信息整合
print("📝 测试 3.2: 多文件信息整合")
file_urls = [TEST_PDF_URL, TEST_PDF_URL]
response = chat_with_files(
    file_urls,
    "整合这两份文档的信息，生成一份综合报告"
)
print_response(response, "多文件信息整合")

## 4. 文档摘要生成测试

In [ ]:
# 测试 4.1: 简短摘要
print("📝 测试 4.1: 简短摘要（100字以内）")
response = chat_with_files(
    TEST_PDF_URL,
    "请用100字以内总结这份文档的核心内容"
)
print_response(response, "简短摘要")

In [ ]:
# 测试 4.2: 详细摘要
print("📝 测试 4.2: 详细摘要")
response = chat_with_files(
    TEST_PDF_URL,
    "请为这份文档生成详细的摘要，包括：\n1. 文档主题\n2. 主要观点\n3. 重要结论"
)
print_response(response, "详细摘要")

In [ ]:
# 测试 4.3: 执行摘要（适合决策者）
print("📝 测试 4.3: 执行摘要")
response = chat_with_files(
    TEST_PDF_URL,
    "请为这份文档生成一份执行摘要，突出对决策者有价值的要点"
)
print_response(response, "执行摘要")

## 5. 文档问答测试

In [ ]:
# 测试 5.1: 事实性问题
print("📝 测试 5.1: 事实性问题")
response = chat_with_files(
    TEST_PDF_URL,
    "这份文档中提到的主要内容是什么？"
)
print_response(response, "事实性问答")

In [ ]:
# 测试 5.2: 推理性问题
print("📝 测试 5.2: 推理性问题")
response = chat_with_files(
    TEST_PDF_URL,
    "根据文档内容，可以得出什么结论或建议？"
)
print_response(response, "推理性问答")

In [ ]:
# 测试 5.3: 假设性问题
print("📝 测试 5.3: 假设性问题")
response = chat_with_files(
    TEST_PDF_URL,
    "如果这份文档的信息用于实际应用，可能会有哪些场景？"
)
print_response(response, "假设性问答")

In [ ]:
# 测试 5.4: 多轮对话模拟
print("📝 测试 5.4: 多轮问答（单轮模拟）")
questions = [
    "这份文档的第一部分内容是什么？",
    "文档中提到了哪些关键点？",
    "这些关键点之间有什么联系？"
]

for i, q in enumerate(questions, 1):
    print(f"\n--- 问题 {i}: {q} ---")
    response = chat_with_files(TEST_PDF_URL, q)
    if "choices" in response:
        print(response["choices"][0]["message"]["content"])
    else:
        print(f"错误: {response}")

## 6. 信息提取测试

In [ ]:
# 测试 6.1: 关键词提取
print("📝 测试 6.1: 关键词提取")
response = chat_with_files(
    TEST_PDF_URL,
    "请从文档中提取5-10个最重要的关键词，并简要说明每个词的重要性"
)
print_response(response, "关键词提取")

In [ ]:
# 测试 6.2: 结构化信息提取（JSON格式）
print("📝 测试 6.2: 结构化信息提取（JSON格式）")
response = chat_with_files(
    TEST_PDF_URL,
    """请将文档的关键信息提取为JSON格式，包含以下字段：
{
  "title": "文档标题",
  "main_topics": ["主题1", "主题2"],
  "key_points": ["要点1", "要点2"],
  "conclusion": "结论"
}
仅返回JSON，不要有其他说明"""
)
print_response(response, "结构化信息提取")

In [ ]:
# 测试 6.3: 特定字段提取
print("📝 测试 6.3: 特定字段提取")
response = chat_with_files(
    TEST_PDF_URL,
    "文档中提到了哪些日期、数字或统计数据？请列出"
)
print_response(response, "特定字段提取")

In [ ]:
# 测试 6.4: 表格数据提取
print("📝 测试 6.4: 表格数据提取")
response = chat_with_files(
    TEST_PDF_URL,
    "如果文档中有表格，请提取表格内容并以Markdown格式呈现"
)
print_response(response, "表格数据提取")

## 7. 不同格式文件测试

In [ ]:
# 测试 7.1: PDF 格式
print("📝 测试 7.1: PDF 格式")
response = chat_with_files(
    TEST_PDF_URL,
    "这是什么类型的文档？请分析其内容"
)
print_response(response, "PDF 格式测试")

In [ ]:
# 测试 7.2: 尝试 TXT 格式（如果可用）
print("📝 测试 7.2: TXT 格式（如果测试文件可用）")
print("注意：需要提供一个可访问的 TXT 文件 URL")

# 示例 TXT URL（需要替换为实际可用的 URL）
TEST_TXT_URL = "https://example.com/sample.txt"  # 请替换为实际 URL

# 如果使用示例 URL，会预期失败
if "example.com" in TEST_TXT_URL:
    print("⚠️ 请替换 TEST_TXT_URL 为实际的 TXT 文件 URL")
else:
    response = chat_with_files(TEST_TXT_URL, "请分析这份TXT文档的内容")
    print_response(response, "TXT 格式测试")

## 8. 错误处理测试

In [ ]:
# 测试 8.1: 无效文件 URL
print("📝 测试 8.1: 无效文件 URL")
invalid_url = "https://invalid-domain-12345.com/nonexistent.pdf"
response = chat_with_files(invalid_url, "请分析这份文档")
print_response(response, "无效 URL 测试")

In [ ]:
# 测试 8.2: 404 文件
print("📝 测试 8.2: 404 文件")
not_found_url = "https://cdn.bigmodel.cn/static/demo/nonexistent_file_12345.pdf"
response = chat_with_files(not_found_url, "请分析这份文档")
print_response(response, "404 文件测试")

In [ ]:
# 测试 8.3: 不支持的模型
print("📝 测试 8.3: 不支持的模型（使用 glm-4-flash）")
response = chat_with_files(
    TEST_PDF_URL,
    "请分析这份文档",
    model="glm-4-flash"  # 其他模型不支持 file_url
)
print_response(response, "不支持模型测试")

In [ ]:
# 测试 8.4: 空文件列表
print("📝 测试 8.4: 空问题测试")
response = chat_with_files(TEST_PDF_URL, "")
print_response(response, "空问题测试")

In [ ]:
# 测试 8.5: 超长问题
print("📝 测试 8.5: 超长问题")
long_question = "请分析" + "详细" * 500 + "这份文档"
response = chat_with_files(TEST_PDF_URL, long_question)
print_response(response, "超长问题测试")

## 9. 性能和限制测试

In [ ]:
# 测试 9.1: 响应时间测试
import time

print("📝 测试 9.1: 响应时间测试")
start_time = time.time()
response = chat_with_files(TEST_PDF_URL, "总结文档内容")
elapsed_time = time.time() - start_time

print(f"⏱️ 响应时间: {elapsed_time:.2f} 秒")
print_response(response, "响应时间测试")

In [ ]:
# 测试 9.2: Token 使用分析
print("📝 测试 9.2: Token 使用分析")

questions = [
    "总结",
    "请详细分析这份文档的所有内容",
    "用一段话总结，用英文回答"
]

for q in questions:
    response = chat_with_files(TEST_PDF_URL, q)
    if "usage" in response:
        usage = response["usage"]
        print(f"问题长度: {len(q)} 字 -> Token: {usage}")
    else:
        print(f"问题: {q[:20]}... -> 错误")

## 10. 总结

In [ ]:
print("""
🎯 文件理解功能测试总结
========================

✅ 已完成测试项:
1. 环境配置 - API 连接和基础函数
2. 单文件理解 - PDF 文档分析
3. 多文件对比 - 多文件输入支持
4. 文档摘要 - 各种摘要类型
5. 文档问答 - 基于内容的问答
6. 信息提取 - 关键词、结构化数据
7. 多格式支持 - PDF、TXT 等
8. 错误处理 - 异常情况处理
9. 性能测试 - 响应时间和 Token 使用

📋 关键发现:
- 仅 GLM-4.6V-Flash 支持文件理解
- 不能与图片/视频同时传入
- 最多支持 50 个文件
- PDF 单文件最大 50MB

⚠️ 使用限制:
- 文件 URL 必须公开可访问
- 不支持同时传入多模态内容
- 扫描件 PDF 需确保文字可识别
""")

print(f"\n测试完成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")